In [21]:
import I2MC
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [3]:
opt = dict()
# General variables for eye-tracking data
# maximum value of horizontal resolution in pixels
opt['xres'] = 1920.0
opt['yres'] = 1080.0  # maximum value of vertical resolution in pixels
# missing value for horizontal position in eye-tracking data (example data uses -xres). used throughout
# internal_helpers as signal for data loss
opt['missingx'] = -opt['xres']
# missing value for vertical position in eye-tracking data (example data uses -yres). used throughout
# internal_helpers as signal for data loss
opt['missingy'] = -opt['yres']
# sampling frequency of data (check that this value matches with values actually obtained from measurement!)
opt['freq'] = 250.0

# Variables for the calculation of visual angle
# These values are used to calculate noise measures (RMS and BCEA) of
# fixations. The may be left as is, but don't use the noise measures then.
# If either or both are empty, the noise measures are provided in pixels
# instead of degrees.
# screen size in cm
opt['scrSz'] = [55.0, 32.5]
# distance to screen in cm.
opt['disttoscreen'] = 65.0

# STEFFEN INTERPOLATION
# max duration (s) of missing values for interpolation to occur
opt['windowtimeInterp'] = 0.1
# amount of data (number of samples) at edges needed for interpolation
opt['edgeSampInterp'] = 2
# maximum displacement during missing for interpolation to be possible
opt['maxdisp'] = opt['xres'] * 0.2 * np.sqrt(2)

# # K-MEANS CLUSTERING
# time window (s) over which to calculate 2-means clustering (choose value so that max. 1 saccade can occur)
opt['windowtime'] = 0.2
# time window shift (s) for each iteration. Use zero for sample by sample processing
opt['steptime'] = 0.02
# maximum number of errors allowed in k-means clustering procedure before proceeding to next file
opt['maxerrors'] = 100
opt['downsamples'] = [2, 5, 10]
# use chebychev filter when down sampling? 1: yes, 0: no. requires signal processing toolbox. is what matlab's
# down sampling internal_helpers do, but could cause trouble (ringing) with the hard edges in eye-movement data
opt['downsampFilter'] = False

# # FIXATION DETERMINATION
# number of standard deviations above mean k-means weights will be used as fixation cutoff
opt['cutoffstd'] = 2.0
# number of MAD away from median fixation duration. Will be used to walk forward at fixation starts and backward at
# fixation ends to refine their placement and stop algorithm from eating into saccades
opt['onoffsetThresh'] = 3.0
# maximum Euclidean distance in pixels between fixations for merging
opt['maxMergeDist'] = 40.0
# maximum time in ms between fixations for merging
opt['maxMergeTime'] = 60.0
# minimum fixation duration after merging, fixations with shorter duration are removed from output
opt['minFixDur'] = 90.0

In [22]:
df_behavioral = pd.read_csv("./data/filteredData/filtered_data.csv")
df_fixation = pd.DataFrame([], columns=["Participant", "Algorithm", "Behavioral", "StartTime", "EndTime", "Duration", "IsOutlier", "SkillScore",
                                        "Fixation_startT", "Fixation_endT",  "Fixation_x", "Fixation_y", "Fixation_x_range", "Fixation_y_range"])
#iterate through each row
for index, row in tqdm(df_behavioral.iterrows(), total=len(df_behavioral)):
    #print("{}-{}: Fixation Calculation".format(row["Participant"], row["Algorithm"]))
    df_eyetracking = pd.read_csv(row["Eyetracking"])
    df_eyetracking["time"] = df_eyetracking["time"].astype(float)
    df_eyetracking["time"] = df_eyetracking["time"] - df_eyetracking["time"].iloc[0]
    # drop col
    df_eyetracking = df_eyetracking.drop(columns=["l_gaze_point_in_user_coordinate_system_x",
                                                  "l_gaze_point_in_user_coordinate_system_y",
                                                  "l_gaze_point_in_user_coordinate_system_z",
                                                  "r_gaze_point_in_user_coordinate_system_x",
                                                  "r_gaze_point_in_user_coordinate_system_y",
                                                  "r_gaze_point_in_user_coordinate_system_z",
                                                  "l_gaze_origin_in_user_coordinate_system_x",
                                                  "l_gaze_origin_in_user_coordinate_system_y",
                                                  "l_gaze_origin_in_user_coordinate_system_z",
                                                  "r_gaze_origin_in_user_coordinate_system_x",
                                                  "r_gaze_origin_in_user_coordinate_system_y",
                                                  "r_gaze_origin_in_user_coordinate_system_z"])

    df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].astype(float) * opt["xres"]
    df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].astype(float) * opt["yres"]
    df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].astype(float) * opt["xres"]
    df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].astype(float) * opt["yres"]
    df_eyetracking["l_valid"] = df_eyetracking["l_valid"].astype(int)
    df_eyetracking["r_valid"] = df_eyetracking["r_valid"].astype(int)

    df_eyetracking["l_miss_x"] = df_eyetracking.apply(lambda row: row["l_display_x"] < -opt["xres"] or row["l_display_x"] > 2 * opt["xres"], axis=1)
    df_eyetracking["l_miss_y"] = df_eyetracking.apply(lambda row: row["l_display_y"] < -opt["yres"] or row["l_display_y"] > 2 * opt["yres"], axis=1)
    df_eyetracking["r_miss_x"] = df_eyetracking.apply(lambda row: row["r_display_x"] < -opt["xres"] or row["r_display_x"] > 2 * opt["xres"], axis=1)
    df_eyetracking["r_miss_y"] = df_eyetracking.apply(lambda row: row["r_display_y"] < -opt["yres"] or row["r_display_y"] > 2 * opt["yres"], axis=1)

    df_eyetracking["l_miss"] = df_eyetracking.apply(lambda row: row["l_miss_x"] or row["l_miss_y"] or not row["l_valid"] >= 1, axis=1)
    df_eyetracking["r_miss"] = df_eyetracking.apply(lambda row: row["r_miss_x"] or row["r_miss_y"] or not row["r_valid"] >= 1, axis=1)

    df_eyetracking.loc[df_eyetracking["l_miss"], "l_display_x"] = opt["missingx"]
    df_eyetracking.loc[df_eyetracking["l_miss"], "l_display_y"] = opt["missingy"]
    df_eyetracking.loc[df_eyetracking["r_miss"], "r_display_x"] = opt["missingx"]
    df_eyetracking.loc[df_eyetracking["r_miss"], "r_display_y"] = opt["missingy"]

    df_eyetracking = df_eyetracking.drop(columns=["l_miss_x", "l_miss_y", "r_miss_x", "r_miss_y", "l_miss", "r_miss"])

    df_eyetracking.rename(columns={"l_display_x": "L_X",
                                   "l_display_y": "L_Y",
                                   "r_display_x": "R_X",
                                   "r_display_y": "R_Y",
                                   "l_valid" : "LValidity",
                                   "r_valid" : "RValidity"}, inplace=True)

    df_eyetracking["time"] = df_eyetracking["time"].astype(float) * 1000.0

    data = {}
    data["L_X"] = df_eyetracking["L_X"].to_numpy()
    data["L_Y"] = df_eyetracking["L_Y"].to_numpy()
    data["R_X"] = df_eyetracking["R_X"].to_numpy()
    data["R_Y"] = df_eyetracking["R_Y"].to_numpy()
    data["LValidity"] = df_eyetracking["LValidity"].to_numpy()
    data["RValidity"] = df_eyetracking["RValidity"].to_numpy()
    data["time"] = df_eyetracking["time"].to_numpy()
    # rename columns
    try:
        fix, data, par = I2MC.I2MC(data, opt, logging=False)
    except Exception as e:
        print('\t\tError in file: {}'.format(e))
        continue

    if not fix:
        print('{}-{}: Fixation calculation had some Problem'.format(row["Participant"], row["Algorithm"]))
        continue


    #f = I2MC.plot.plot_data_and_fixations(data, fix, fix_as_line=True, res=[opt['xres'], opt['yres']])

    participant = row["Participant"]
    algorithm = row["Algorithm"]
    behavioral = row["Behavioral"]
    start_time = row["StartTime"]
    end_time = row["EndTime"]
    duration = row["Duration"]
    is_outlier = row["IsOutlier"]
    skill_score = row["SkillScore"]

    fixations_start_time = np.array(fix["startT"])
    fixations_end_time = np.array(fix["endT"])
    fixations_x_pos = np.array(fix["xpos"])
    fixations_y_pos = np.array(fix["ypos"])
    fixations_x_range = np.array(fix["fixRangeX"])
    fixations_y_range = np.array(fix["fixRangeY"])

    df_fixation.loc[len(df_fixation)] = [participant, algorithm, behavioral, start_time, end_time, duration, is_outlier, skill_score,
                                         fixations_start_time, fixations_end_time, fixations_x_pos, fixations_y_pos, fixations_x_range, fixations_y_range]

  0%|          | 0/1072 [00:00<?, ?it/s]

C:\Users\jonas\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\jonas\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\jonas\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
C:\Users\jonas\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\jonas\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\jonas\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis,

In [24]:
df_fixation.to_csv("./data/filteredData/fixation_stats.csv", index=False, sep=";")

In [25]:
df_fixation

,Participant,Algorithm,Behavioral,StartTime,EndTime,Duration,IsOutlier,SkillScore,Fixation_startT,Fixation_endT,Fixation_x,Fixation_y,Fixation_x_range,Fixation_y_range
0,1,IsPrime,Right,0.000000,12.390280,12.390280,False,0.331385,"[0.0, 256.0070000000003, 756.0169999999999, 11...","[232.00299999999885, 732.016999999999, 1112.02...","[938.5366058349608, 1015.9948253631592, 1111.4...","[455.2750360965729, 458.4717518091202, 454.982...","[0.47429750865000636, 1.0301863858825435, 1.93...","[1.2040604086298201, 1.3868055259614287, 0.922..."
1,1,SiebDesEratosthenes,Wrong,49.778238,202.350152,152.571914,False,0.331385,"[60.00500000000386, 240.01000000000516, 756.02...","[212.01100000000395, 732.0250000000001, 1484.0...","[868.4300708770751, 1007.4270915985107, 1058.0...","[291.24348163604736, 299.3649584054947, 298.56...","[0.4899310864185897, 1.254163881639752, 1.3195...","[0.9883541625518906, 1.0339219899553656, 1.154..."
2,1,IsAnagram,Right,254.122970,363.738694,109.615724,False,0.331385,"[55.9989999999857, 216.0069999999905, 816.0280...","[192.00799999998708, 792.0280000000162, 1004.0...","[915.1887130737305, 873.2017993927002, 872.125...","[202.8214788436889, 209.60132539272308, 257.19...","[0.388032176379905, 0.7063182674207252, 0.3624...","[1.3295969012258404, 1.2773122602118223, 1.001..."
3,1,RemoveDoubleChar,Right,401.629505,455.085781,53.456276,False,0.331385,"[0.0, 144.00299999999788, 316.0090000000082, 7...","[128.00099999998338, 292.01000000000477, 708.0...","[755.0806045532227, 869.5891571044922, 917.799...","[289.3947207927704, 287.10810273885727, 292.48...","[0.5601077254451202, 0.5157867022364909, 1.290...","[1.3639902214825115, 1.6714170676586815, 1.406..."
4,1,BinToDecimal,Right,555.252914,605.175005,49.922091,False,0.331385,"[0.0, 152.00400000003356, 332.012000000077, 55...","[136.00500000006832, 316.0120000000006, 532.01...","[813.7207317352295, 857.8827238082886, 908.068...","[442.7486264705658, 433.9537274837494, 438.147...","[0.45957416183140104, 0.5936237293531322, 0.66...","[1.0595384895475222, 2.8838721218599566, 1.571..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,71,DumpSorting,Wrong,2913.891198,3027.260143,113.368945,False,0.435651,"[0.0, 148.00300000024436, 476.0150000001886, 8...","[112.00300000018615, 452.01500000030137, 828.0...","[784.2814636230469, 843.5760498046875, 984.981...","[402.68189549446106, 135.4314742237329, 124.32...","[0.6281684387157223, 1.1857159590126556, 1.007...","[1.469294393096719, 1.1084554284290438, 0.9716..."
1068,71,BinomialCoefficient,Right,3138.411363,3189.049224,50.637861,False,0.435651,"[0.0, 180.00200000005862, 496.0129999999481, 7...","[152.00200000026598, 472.01300000006086, 712.0...","[898.8488101959227, 995.0553131103516, 911.907...","[380.1622349023819, 376.2835192680359, 381.379...","[0.43449657675406805, 0.37841623655838225, 0.5...","[0.731904516281904, 0.9440338391736062, 1.1622..."
1069,71,IsAnagram,Right,3227.575716,3338.571470,110.995754,False,0.435651,"[36.00000000005821, 260.00900000008187, 624.02...","[232.0089999998345, 604.0250000000924, 812.030...","[901.2126016616821, 849.427900314331, 774.6641...","[183.968012407422, 157.75504857301712, 182.355...","[0.4171942422438695, 0.38097427449001864, 0.61...","[1.1478479585254198, 1.5037924000714649, 1.084..."
1070,71,ArrayAverage,Right,3375.662381,3383.972645,8.310264,False,0.435651,"[0.0, 176.00599999968836, 884.0259999997215, 1...","[152.00599999980113, 868.0239999998776, 1072.0...","[876.8252563476561, 1010.4466152191162, 884.08...","[337.34915792942047, 341.4681378006935, 445.61...","[0.4270001747160645, 1.7542686049381004, 0.428...","[1.4156344500067244, 2.7952511832288627, 0.985..."


In [26]:
df_fixation["NumberOfFixations"] = df_fixation.apply(lambda row: len(row["Fixation_startT"]), axis=1)